In [53]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
import seaborn as sns
from math import pi
import matplotlib.pyplot as plt
import random
%matplotlib inline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate,KFold, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from xgboost import XGBClassifier

train_labels = pd.read_csv('Archivos de datos/train_labels.csv', encoding='latin-1')
train_values = pd.read_csv('Archivos de datos/train_values.csv', encoding='latin-1')

join = train_values.merge(train_labels)

## Experimentacion

join.columns

join['damage_grade'].value_counts()

X=pd.get_dummies(join.loc[:,:'has_secondary_use_other'])
y=join['damage_grade'].astype(int)

## Creamos set de entrenamiento y test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=1)

In [11]:
%%time
# RandomForest
## Estandard
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.540187
Wall time: 1min 12s


In [12]:
%%time
# RandomForest
## Mejores hiperparametros
rf_model = RandomForestClassifier(n_estimators=80, min_samples_split=6, min_samples_leaf=4, max_depth=18)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.562698
Wall time: 39 s


In [36]:
## Busqueda de los mejores Hiperparametros con Random
n_estimators=np.arange(10,110,10)
max_depth=np.arange(2,22,4)
min_samples_split=np.arange(1,11,1)
min_samples_leaf=np.arange(1,11,1)

param_grid={'n_estimators':n_estimators,'max_depth':max_depth,'min_samples_split':min_samples_split,'min_samples_leaf':min_samples_leaf}

clf=RandomForestClassifier()

kf=KFold(n_splits=2,shuffle=True)

rs=RandomizedSearchCV(clf,param_distributions=param_grid,cv=kf,scoring='f1_micro')

In [37]:
%%time
rs.fit(X,y)

C:\Users\ezequ\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\ezequ\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ezequ\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\ezequ\anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\ezequ\anaconda3\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\ezequ\anaconda3\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

Wall time: 2min 49s


RandomizedSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=True),
                   estimator=RandomForestClassifier(),
                   param_distributions={'max_depth': array([ 2,  6, 10, 14, 18]),
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_samples_split': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100])},
                   scoring='f1_micro')

In [38]:
rs.best_params_

{'n_estimators': 90,
 'min_samples_split': 6,
 'min_samples_leaf': 4,
 'max_depth': 14}

In [ ]:
#Con Grid

In [45]:
pipe = make_pipeline(StandardScaler(), 
                     RandomForestClassifier(random_state=2018))
pipe

param_grid = {'randomforestclassifier__n_estimators': [5, 100],
              'randomforestclassifier__min_samples_leaf': [1, 12],
              'randomforestclassifier__min_samples_split': [2, 12],
              'randomforestclassifier__max_depth': [1, 25]
             }

gs = GridSearchCV(pipe, param_grid, cv=5)


In [46]:
%%time
gs.fit(X,y)

Wall time: 18min 19s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(random_state=2018))]),
             param_grid={'randomforestclassifier__max_depth': [1, 25],
                         'randomforestclassifier__min_samples_leaf': [1, 12],
                         'randomforestclassifier__min_samples_split': [2, 12],
                         'randomforestclassifier__n_estimators': [5, 100]})

In [47]:
gs.best_params_

{'randomforestclassifier__max_depth': 25,
 'randomforestclassifier__min_samples_leaf': 1,
 'randomforestclassifier__min_samples_split': 2,
 'randomforestclassifier__n_estimators': 100}

In [ ]:
#---------------------------------------------------------------------------------------------------------

In [54]:
clf=RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1, max_depth=25)
clf.fit(X,y)

RandomForestClassifier(max_depth=25)

In [55]:
X_test=pd.read_csv('Archivos de datos/test_values.csv')
X_test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [56]:
prediction=clf.predict((pd.get_dummies(X_test)))
result=pd.DataFrame(prediction)

In [57]:
result['building_id']=X_test['building_id']
result.rename(columns={0:'damage_grade'},inplace=True)
result=result[['building_id','damage_grade']]
result.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3


In [58]:
result.to_csv('Archivos de datos/resultado.csv',index=False)